In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [19]:
import pandas as pd
w_df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/weather_2012.csv')

Data transformation and Data cleaning


In [20]:
Q1=w_df['Temp (C)'].quantile(0.25)
Q3 = w_df['Temp (C)'].quantile(0.75)
IQR = Q1-Q3
lower_bound = Q1-1.5*IQR
Upper_bound = Q3+1.5*IQR
#Any temperature between 28.15 or -9.24 considered as outlier
w_df['is_outliers'] = (w_df['Temp (C)']<lower_bound) | (w_df['Temp (C)']>28.15)
#we can identify the outliers

'''Filtering the data'''


In [5]:
#checking the temp between 10 and 25 , weather condition is cloudy or clear and windspeed is above 20
df_filter= w_df[((w_df['Temp (C)']>10) & (w_df['Temp (C)']<25)) & ((w_df['Weather'] == "Cloudy") | (w_df['Weather'] == "Clear")) & (
            w_df['Wind Spd (km/h)'] > 20)]

In [6]:
#filtering based on string contains rainy or snow using str.contains
df_snow =w_df[w_df["Weather"].str.contains('Rain|Snow')]

'''Date and time operations'''

In [21]:
w_df['Date/Time'] = pd.to_datetime(w_df['Date/Time']) #changed to datetime datatype
w_df = w_df.set_index('Date/Time')


In [ ]:
#caliculating average temp for every week
df_resample = w_df.resample('W').agg({'Temp (C)':'mean'})
df_resample

In [23]:
df_avg_temp = w_df.resample('D').agg({'Temp (C)':'mean'})

In [24]:
#caliculating the daily temperature difference and adding it as new column
df_daily_temp_diff = df_avg_temp.diff()
w_df['temp_diff']=df_daily_temp_diff

In [25]:
w_df = w_df.reset_index() #since in previous cell i set the datatime as index

Grouping the data by weather condition and month

In [26]:
w_df["Month"]=w_df['Date/Time'].dt.month

In [ ]:
w_df.groupby(['Weather','Month']).agg({'Temp (C)':'mean', 'Wind Spd (km/h)':'max'})

For each weather condition, calculate:

The mean temperature, but only for days where the wind speed was below 15 km/h.
The maximum temperature during rainy days.

In [29]:
w_df.rename(columns={'Wind Spd (km/h)':'Wind Spd'},inplace= True)#renaming for convience


In [49]:
def mean_temp(df):
  return df[df['Wind Spd']<15]['Temp (C)'].mean()
def max_temp(df):# using str.contain() because weather column have values like frezing rain,heavy rain so cannot =="rain"
  return df[df['Weather'].str.contains('Rain')]['Temp (C)'].max()

In [ ]:
w_df.groupby('Weather').apply(lambda x: (mean_temp(x), max_temp(x))) # Applying both functions together to each group

Merging and splitting dataframes

Spliting the dataframe by 6 months and joining

In [58]:
w_df['month']=w_df['Date/Time'].dt.month

In [61]:
df1 = w_df[w_df['month']<=6]
df2= w_df[w_df['month']>6]

In [ ]:
new_df = pd.concat([df1,df2])
new_df

In [ ]:
df3 = w_df[w_df['Weather'].str.contains('Fog')]
df3

In [ ]:
new_df2 = pd.concat([w_df, df3]).drop_duplicates()
new_df2

In [92]:
w_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date/Time           8784 non-null   datetime64[ns]
 1   Temp (C)            8784 non-null   float64       
 2   Dew Point Temp (C)  8784 non-null   float64       
 3   Rel Hum (%)         8784 non-null   int64         
 4   Wind Spd            8784 non-null   int64         
 5   Visibility (km)     8784 non-null   float64       
 6   Stn Press (kPa)     8784 non-null   float64       
 7   Weather             8784 non-null   object        
 8   is_outliers         8784 non-null   bool          
 9   temp_diff           365 non-null    float64       
 10  Month               8784 non-null   int32         
 11  month               8784 non-null   int32         
dtypes: bool(1), datetime64[ns](1), float64(5), int32(2), int64(2), object(1)
memory usage: 695.0+ KB


Rolling window and moving averages

In [ ]:
w_df_rolling = w_df['Temp (C)'].rolling(window=7).mean()# caliculating 7 day rolling average temp
w_df_rolling.head(40)

In [ ]:
w_df_std = w_df['Wind Spd'].rolling(window= 30).std()#caliculating 30 day rolling standard deviation of wind speed
import matplotlib.pyplot as plt
plt.plot(w_df_std)

Using pivot concept
Creating a  pivot table that shows the average temperature for each weather condition, grouped by month.



In [ ]:
pivot_df = pd.pivot_table(w_df,
values = 'Temp (C)',
index = 'Date/Time',
columns = 'Weather',
aggfunc = 'mean'

)
pivot_df

In [105]:
w_df['week'] = w_df['Date/Time'].dt.isocalendar().week

In [ ]:
pd.crosstab(w_df['Weather'],w_df['week']) #how frequently each weather condition occurred on each day of the week.